In [1]:
# Import the above libraries

import streamlit as st
import requests
from bs4 import BeautifulSoup
import os
import google.generativeai as genai

/opt/conda/envs/r/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import socket
import re

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

# Cloud project id.
PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]  # @param {type:"string"}

# The region you want to launch jobs in.
PREFIX_NUM_ONLY=int(str(re.search(r'\d+', UNIQUE_PREFIX).group()))
REGION_ALLOCATE=PREFIX_NUM_ONLY%3
if REGION_ALLOCATE == 0:
    REGION = "asia-southeast1"
elif REGION_ALLOCATE == 1:
    REGION = "us-central1"
else:
    REGION = "europe-west4"

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = f"{PROJECT_ID}-{UNIQUE_PREFIX}"  # @param {type:"string"} 
BUCKET_URI = f"gs://{GCS_BUCKET}"  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = !(gcloud config get-value core/account)  # @param {type:"string"}
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]

! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={REGION}
! pip install -q gdown

Creating gs://my-project-0004-346516-pytorch112kagglewbi/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
# Initialize the Google API key and import Gemini-pro model.

st.title("Proposal Calls") # Title for the page

API_KEY = '<<< ADD your OWN key-test'

genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-1.5-pro-001")


model = genai.GenerativeModel('gemini-pro')

### Create a function read_input() to extract raw data from the site. And then to feed it to the model as a prompt to structure the data.


In [24]:

def read_input():
  # dictionary of all the links to be webscraped.
  # You can add more if you want to
    links = {
       "1":['@vkmauryavk','https://medium.com/'], #["DST","https://dst.gov.in/call-for-proposals"],
       "2":['@vkmauryavk','https://medium.com/']
    }
    for i in range(1,3):
       url = links[str(i)][1] # Get URL of each organization
       r = requests.get(url) # Request for data
       soup = BeautifulSoup(r.text, 'html.parser') # Parse the HTML elements
       data = soup.text # Get raw data in string format
       link = soup.find_all('a', href=True) # Get list of all links on the site in html formet
       l = ""
       for a in link:
           l = l +"\n"+ a['href'][1:] # Get the actual links
      # Create a query
       query = data + "name of organization is"+links[str(i)][0]+ "Jumbled links of calls for proposals:"+l+"\n Create a table with the following columns: Call for proposals or joint call for proposals along with respective link, opening date, closing date and the name of the organization."
       output = llm_function(query)
    print('output',output)
    return(output)

In [25]:
# Create another function llm_function() to generate the response.

def llm_function(query):
    response = model.generate_content(query) # Generate response
    st.markdown(response.text) # Print it out using streamlit

In [26]:
# Call the main function.

# if __name__ == __main__:
read_input()

output None
